In [1]:
# cell 03
import numpy as np                                # For matrix operations and numerical processing
import pandas as pd                               # For munging tabular data
import matplotlib.pyplot as plt                   # For charts and visualizations
from IPython.display import Image                 # For displaying images in the notebook
from IPython.display import display               # For displaying outputs in the notebook
from time import gmtime, strftime                 # For labeling SageMaker models, endpoints, etc.
import sys                                        # For writing outputs to notebook
import math                                       # For ceiling function
import json                                       # For parsing hosting outputs
import os                                         # For manipulating filepath names
import sagemaker 
import zipfile     # Amazon SageMaker's Python SDK provides many helper functions

/opt/conda/lib/python3.12/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
test_id = pd.read_csv('test_data/(Test)ID_Data_202501.csv')
test_accts = pd.read_csv('test_data/(Test)ACCTS_Data_202501.csv')
test_sav_txn  = pd.read_csv('test_data/(Test)SAV_TXN_Data_202501.csv')

In [3]:
# 以CUST_ID和ACCT_NBR為鍵合併帳戶資料與客戶資料
test_df = pd.merge(test_accts, test_id, on='CUST_ID', how='left')

# 合併交易資料
test_df = pd.merge(test_df, test_sav_txn, on=['ACCT_NBR', 'CUST_ID'], how='left')
test_df

,ACCT_NBR,CUST_ID,CANCEL_NO_CONTACT,IS_DIGITAL,ACCT_OPEN_DT,AUM_AMT,DATE_OF_BIRTH,YEARLYINCOMELEVEL,CNTY_CD,TX_DATE,...,OWN_TRANS_ID,CHANNEL_CODE,TRN_CODE,BRANCH_NO,EMP_NO,mb_check,eb_check,SAME_NUMBER_IP,SAME_NUMBER_UUID,DAY_OF_WEEK
0,ACCT20379,ID19286,0,0,15758,142508,7,25.0,12.0,18299,...,ID99999,13,42,B63,E4957,0,0,0,0,Friday
1,ACCT5965,ID5583,0,0,15715,2218985,57,126.0,12.0,18291,...,ID99999,16,41,B111,E4956,0,0,0,0,Thursday
2,ACCT5965,ID5583,0,0,15715,2218985,57,126.0,12.0,18299,...,ID99999,16,41,B111,E4956,0,0,0,0,Friday
3,ACCT23936,ID22722,0,0,16339,3623,25,25.0,12.0,18298,...,ID99999,16,41,B111,E4956,0,0,0,0,Thursday
4,ACCT12076,ID11435,0,0,16237,173790,70,25.0,12.0,18298,...,ID99999,13,42,B63,E4956,0,0,0,0,Thursday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52117,ACCT3921,ID3683,0,1,16016,71,27,25.0,12.0,18308,...,ID99999,17,30,B111,E4795,0,0,0,0,Sunday
52118,ACCT3921,ID3683,0,1,16016,71,27,25.0,12.0,18308,...,ID99999,17,30,B111,E4535,0,0,0,0,Sunday
52119,ACCT3921,ID3683,0,1,16016,71,27,25.0,12.0,18308,...,ID99999,17,27,B111,E4779,0,0,0,0,Sunday
52120,ACCT3921,ID3683,0,1,16016,71,27,25.0,12.0,18311,...,ID99999,16,20,B105,E4956,2,0,0,0,Wednesday


In [10]:
# 2. 數值欄位轉型
time_columns = ['TX_DATE', 'ACCT_OPEN_DT','TX_TIME','DATE_OF_BIRTH']
for col in time_columns:
    test_df[col] = pd.to_numeric(test_df[col], errors='coerce').fillna(0)

# 3. 標記需要 one-hot 的類別欄位（不含識別欄位）
categorical_cols = [
    'CHANNEL_CODE', 'TRN_CODE', 'DAY_OF_WEEK', 'CNTY_CD', 'YEARLYINCOMELEVEL'
]

# 4. one-hot 編碼（保留識別欄位）
data_encoded = pd.get_dummies(test_df, columns=categorical_cols, dtype=float)
# 4. 排除識別欄位
id_cols = ['CUST_ID', 'ACCT_NBR','OWN_TRANS_ACCT', 'OWN_TRANS_ID', 'BRANCH_NO', 'EMP_NO']
test_features = data_encoded.drop(columns=id_cols, errors='ignore')
test_features

,CANCEL_NO_CONTACT,IS_DIGITAL,ACCT_OPEN_DT,AUM_AMT,DATE_OF_BIRTH,TX_DATE,TX_TIME,DRCR,TX_AMT,PB_BAL,...,CNTY_CD_15.0,YEARLYINCOMELEVEL_25.0,YEARLYINCOMELEVEL_30.0,YEARLYINCOMELEVEL_125.0,YEARLYINCOMELEVEL_126.0,YEARLYINCOMELEVEL_225.0,YEARLYINCOMELEVEL_350.0,YEARLYINCOMELEVEL_400.0,YEARLYINCOMELEVEL_750.0,YEARLYINCOMELEVEL_1000.0
0,0,0,15758,142508,7,18299,2,2,81045,143674,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,15715,2218985,57,18291,0,2,796,569062,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0,0,15715,2218985,57,18299,3,2,1503,570552,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0,0,16339,3623,25,18298,2,2,397,1951,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,16237,173790,70,18298,2,2,54553,383051,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52117,0,1,16016,71,27,18308,14,1,19995,8599,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52118,0,1,16016,71,27,18308,14,1,8029,583,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52119,0,1,16016,71,27,18308,23,1,4,573,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52120,0,1,16016,71,27,18311,16,1,579,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
print(list(test_features.columns))
print(len(test_features.columns))

['CANCEL_NO_CONTACT', 'IS_DIGITAL', 'ACCT_OPEN_DT', 'AUM_AMT', 'DATE_OF_BIRTH', 'TX_DATE', 'TX_TIME', 'DRCR', 'TX_AMT', 'PB_BAL', 'mb_check', 'eb_check', 'SAME_NUMBER_IP', 'SAME_NUMBER_UUID', 'CHANNEL_CODE_2', 'CHANNEL_CODE_3', 'CHANNEL_CODE_4', 'CHANNEL_CODE_6', 'CHANNEL_CODE_7', 'CHANNEL_CODE_8', 'CHANNEL_CODE_9', 'CHANNEL_CODE_10', 'CHANNEL_CODE_11', 'CHANNEL_CODE_12', 'CHANNEL_CODE_13', 'CHANNEL_CODE_14', 'CHANNEL_CODE_15', 'CHANNEL_CODE_16', 'CHANNEL_CODE_17', 'CHANNEL_CODE_18', 'CHANNEL_CODE_19', 'TRN_CODE_1', 'TRN_CODE_3', 'TRN_CODE_4', 'TRN_CODE_6', 'TRN_CODE_7', 'TRN_CODE_8', 'TRN_CODE_9', 'TRN_CODE_10', 'TRN_CODE_11', 'TRN_CODE_12', 'TRN_CODE_14', 'TRN_CODE_15', 'TRN_CODE_16', 'TRN_CODE_17', 'TRN_CODE_18', 'TRN_CODE_20', 'TRN_CODE_21', 'TRN_CODE_22', 'TRN_CODE_23', 'TRN_CODE_24', 'TRN_CODE_25', 'TRN_CODE_26', 'TRN_CODE_27', 'TRN_CODE_28', 'TRN_CODE_29', 'TRN_CODE_30', 'TRN_CODE_31', 'TRN_CODE_33', 'TRN_CODE_35', 'TRN_CODE_36', 'TRN_CODE_37', 'TRN_CODE_38', 'TRN_CODE_40', 'TRN

In [14]:
# # 5. 特徵對齊（必須與訓練時一致）
# # 載入訓練時的特徵欄位順序
train_feature_list = ['CANCEL_NO_CONTACT', 'IS_DIGITAL', 'ACCT_OPEN_DT', 'AUM_AMT', 'DATE_OF_BIRTH', 'TX_DATE', 'TX_TIME', 'DRCR', 'TX_AMT', 'PB_BAL', 'mb_check', 'eb_check', 'SAME_NUMBER_IP', 'SAME_NUMBER_UUID', 'DATA_DT', 'IS_REPORTED', 'CHANNEL_CODE_1', 'CHANNEL_CODE_2', 'CHANNEL_CODE_3', 'CHANNEL_CODE_4', 'CHANNEL_CODE_5', 'CHANNEL_CODE_6', 'CHANNEL_CODE_7', 'CHANNEL_CODE_8', 'CHANNEL_CODE_9', 'CHANNEL_CODE_10', 'CHANNEL_CODE_11', 'CHANNEL_CODE_12', 'CHANNEL_CODE_13', 'CHANNEL_CODE_14', 'CHANNEL_CODE_15', 'CHANNEL_CODE_16', 'CHANNEL_CODE_17', 'CHANNEL_CODE_18', 'CHANNEL_CODE_19', 'TRN_CODE_1', 'TRN_CODE_2', 'TRN_CODE_3', 'TRN_CODE_4', 'TRN_CODE_5', 'TRN_CODE_6', 'TRN_CODE_7', 'TRN_CODE_8', 'TRN_CODE_10', 'TRN_CODE_11', 'TRN_CODE_12', 'TRN_CODE_13', 'TRN_CODE_14', 'TRN_CODE_15', 'TRN_CODE_17', 'TRN_CODE_18', 'TRN_CODE_19', 'TRN_CODE_20', 'TRN_CODE_22', 'TRN_CODE_24', 'TRN_CODE_25', 'TRN_CODE_26', 'TRN_CODE_27', 'TRN_CODE_28', 'TRN_CODE_29', 'TRN_CODE_30', 'TRN_CODE_31', 'TRN_CODE_32', 'TRN_CODE_33', 'TRN_CODE_34', 'TRN_CODE_35', 'TRN_CODE_36', 'TRN_CODE_37', 'TRN_CODE_38', 'TRN_CODE_39', 'TRN_CODE_40', 'TRN_CODE_41', 'TRN_CODE_42', 'TRN_CODE_43', 'TRN_CODE_44', 'TRN_CODE_45', 'TRN_CODE_46', 'TRN_CODE_47', 'TRN_CODE_48', 'TRN_CODE_49', 'TRN_CODE_50', 'TRN_CODE_51', 'TRN_CODE_52', 'TRN_CODE_53', 'TRN_CODE_54', 'DAY_OF_WEEK_Friday', 'DAY_OF_WEEK_Monday', 'DAY_OF_WEEK_Saturday', 'DAY_OF_WEEK_Sunday', 'DAY_OF_WEEK_Thursday', 'DAY_OF_WEEK_Tuesday', 'DAY_OF_WEEK_Wednesday', 'CNTY_CD_1.0', 'CNTY_CD_2.0', 'CNTY_CD_3.0', 'CNTY_CD_4.0', 'CNTY_CD_5.0', 'CNTY_CD_6.0', 'CNTY_CD_7.0', 'CNTY_CD_8.0', 'CNTY_CD_9.0', 'CNTY_CD_11.0', 'CNTY_CD_12.0', 'CNTY_CD_13.0', 'CNTY_CD_14.0', 'YEARLYINCOMELEVEL_25.0', 'YEARLYINCOMELEVEL_30.0', 'YEARLYINCOMELEVEL_125.0', 'YEARLYINCOMELEVEL_126.0', 'YEARLYINCOMELEVEL_225.0', 'YEARLYINCOMELEVEL_350.0', 'YEARLYINCOMELEVEL_400.0', 'YEARLYINCOMELEVEL_750.0', 'YEARLYINCOMELEVEL_1000.0']  # 你的訓練特徵清單
test_features = test_features.reindex(columns=train_feature_list, fill_value=-1)
test_features

,CANCEL_NO_CONTACT,IS_DIGITAL,ACCT_OPEN_DT,AUM_AMT,DATE_OF_BIRTH,TX_DATE,TX_TIME,DRCR,TX_AMT,PB_BAL,...,CNTY_CD_14.0,YEARLYINCOMELEVEL_25.0,YEARLYINCOMELEVEL_30.0,YEARLYINCOMELEVEL_125.0,YEARLYINCOMELEVEL_126.0,YEARLYINCOMELEVEL_225.0,YEARLYINCOMELEVEL_350.0,YEARLYINCOMELEVEL_400.0,YEARLYINCOMELEVEL_750.0,YEARLYINCOMELEVEL_1000.0
0,0,0,15758,142508,7,18299,2,2,81045,143674,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,15715,2218985,57,18291,0,2,796,569062,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0,0,15715,2218985,57,18299,3,2,1503,570552,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0,0,16339,3623,25,18298,2,2,397,1951,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,16237,173790,70,18298,2,2,54553,383051,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52117,0,1,16016,71,27,18308,14,1,19995,8599,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52118,0,1,16016,71,27,18308,14,1,8029,583,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52119,0,1,16016,71,27,18308,23,1,4,573,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52120,0,1,16016,71,27,18311,16,1,579,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# 6. 取出標籤
X_test = test_features.drop(columns=['IS_REPORTED'], errors='ignore').to_numpy(dtype=float)

In [18]:
import pandas as pd
import numpy as np
import boto3
import io

def sagemaker_batch_predict(X_test, endpoint_name, batch_size=500):
    runtime = boto3.client('sagemaker-runtime', region_name='us-west-2')
    all_preds = []
    n = X_test.shape[0]
    for start in range(0, n, batch_size):
        end = min(start + batch_size, n)
        batch = X_test[start:end]
        csv_buffer = io.StringIO()
        pd.DataFrame(batch).to_csv(csv_buffer, header=False, index=False)
        payload = csv_buffer.getvalue()
        response = runtime.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType='text/csv',
            Body=payload
        )
        result = response['Body'].read().decode('utf-8')
        preds = np.fromstring(result, sep=',')
        all_preds.append(preds)
    return np.concatenate(all_preds)

# 使用方式
endpoint_name = 'xgboost-2025-04-27-01-14-48-209'
# 假設 X_test 是 numpy array 或 DataFrame
y_pred = sagemaker_batch_predict(X_test, endpoint_name, batch_size=500)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:29                                                                                   │
│                                                                                                  │
│   26 # 使用方式                                                                                  │
│   27 endpoint_name = 'xgboost-2025-04-27-01-14-48-209'                                           │
│   28 # 假設 X_test 是 numpy array 或 DataFrame                                                   │
│ ❱ 29 y_pred = sagemaker_batch_predict(X_test, endpoint_name, batch_size=500)                     │
│   30                                                                                             │
│                                                                                                  │
│ in sagemaker_batch_predict:16                                                                    │
│                                                                                                  │
│   13 │   │   csv_buffer = io.StringIO()                                                          │
│   14 │   │   pd.DataFrame(batch).to_csv(csv_buffer, header=False, index=False)                   │
│   15 │   │   payload = csv_buffer.getvalue()                                                     │
│ ❱ 16 │   │   response = runtime.invoke_endpoint(                                                 │
│   17 │   │   │   EndpointName=endpoint_name,                                                     │
│   18 │   │   │   ContentType='text/csv',                                                         │
│   19 │   │   │   Body=payload                                                                    │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/client.py:569 in _api_call                      │
│                                                                                                  │
│    566 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    567 │   │   │   │   )                                                                         │
│    568 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  569 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    570 │   │                                                                                     │
│    571 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    572                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/client.py:1023 in _make_api_call                │
│                                                                                                  │
│   1020 │   │   │   │   "Code"                                                                    │
│   1021 │   │   │   )                                                                             │
│   1022 │   │   │   error_class = self.exceptions.from_code(error_code)                           │
│ ❱ 1023 │   │   │   raise error_class(parsed_response, operation_name)                            │
│   1024 │   │   else:                                                                             │
│   1025 │   │   │   return parsed_response                                                        │
│   1026                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ModelError: An error occurred (ModelError) when calling the InvokeEnd